<a href="https://colab.research.google.com/github/Jacquesjh/Audiobook/blob/main/Audiobook_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazendo upload do documento .csv com os dados para serem pré processados

In [ ]:
%cd /content/drive/MyDrive/Machine Learning/Audiobook

/content/drive/MyDrive/Machine Learning/Audiobook


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Audiobooks_data.csv to Audiobooks_data.csv


##Aqui nos importamos todas as bibliotecas que iremos usar para o processo

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import io

##Utilizamos pandas para transformar o arquivo ainda em .csv em um dataframe trabalhável
raw_data = pd.read_csv(io.BytesIO(uploaded['Audiobooks_data.csv']))

##Com as bibliotecas importados, começamos a trabalhar no nosso dataset

In [ ]:
#Aqui apenas visualizamos o nosso dataset
raw_data 

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,4.61,0,8.91,0.00,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14078,28220,1620.0,1620,5.33,5.33,1,9.00,0.61,988.2,0,4,0
14079,28671,1080.0,1080,6.55,6.55,1,6.00,0.29,313.2,0,29,0
14080,31134,2160.0,2160,6.14,6.14,0,8.91,0.00,0.0,0,0,0
14081,32832,1620.0,1620,5.33,5.33,1,8.00,0.38,615.6,0,90,0


In [ ]:
##Como o nosso dataset foi importado para código como um dataframe de pandas, é melhor transforma-lo em um array numpy, pois é mais facil de trabalhar
array_raw_data = raw_data.to_numpy()
array_raw_data

array([[1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       [2.8820e+03, 1.6200e+03, 1.6200e+03, ..., 1.0000e+00, 1.2900e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [ ]:
##Aqui nos separamos nosso dataset em dois: os inputs e os targets do nosso problema
clear_inputs = array_raw_data[: ,1: -1]
clear_targets = array_raw_data[: ,-1]
##Podemos sobrevoar o mouse por cima das variaveis e objetos para verificar suas estruturas

##Agora para equilibrar o nosso dataset, deixando o número de 1s e 0s iguais

In [ ]:
##Queremos contar o numero de targets = 1 e verificar seus indices
num_one_targets = int(np.sum(clear_targets))
num_zero_counter = 0
index_to_remove = []

##Nessa função iremos deixar os targets = 0 até eles se equilibrarem com o numero de targets = 1
##então nos armazenamos todos os outros indices que restam para excluirmos depois dos datasets
for i in range(clear_targets.shape[0]):
  if clear_targets[i] == 0:
    num_zero_counter += 1

    if num_zero_counter > num_one_targets:
      index_to_remove.append(i)

##Aqui nos excluímos os números de dados, que sao apenas de targets = 0 que iriam desequilibrar nossos dados em relação ao targets = 1
balanced_inputs = np.delete(clear_inputs, index_to_remove, axis = 0)
balanced_targets = np.delete(clear_targets, index_to_remove, axis = 0)

##Agora nos queremos padronizar os features do inputs (padronizar cada coluna da matriz inputs)

In [ ]:
scaled_inputs = preprocessing.scale(balanced_inputs)

#Com isso, devemos embaralhar nossos dados, mas devemos fazer isso de modo a preservar os indices de inputs e targets, ou seja, que sejam embaralhados igualmente

In [ ]:
##Aqui nos armazenanos os indices e embaralhamos]
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

#Então nos aplicamos os indices embaralhados aos datasets
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = balanced_targets[shuffled_indices]

#Precisamos, então, dividir nosso dataset em Train, Validation e Test datasets

In [ ]:
#Aqui nos precisamos saber o número de dados que temos no nosso dataset, sendo que o número de dados no inputs é igual ao targets
samples_counts = shuffled_indices.shape[0]

#Queremos que nosso train set seja 80% do total, validation seja 10% e test seja 10%
train_samples = int(0.8*samples_counts)
validation_samples = int(0.1*samples_counts)
test_samples = samples_counts - train_samples - validation_samples

#Agora divimos realmente nosso banco de dados
train_inputs = shuffled_inputs[: train_samples]      ##Ele começa de 0 até o número train_samples
train_targets = shuffled_targets[: train_samples]

validation_inputs = shuffled_inputs[train_samples: train_samples + validation_samples]
validation_targets = shuffled_targets[train_samples: train_samples + validation_samples]

test_inputs = shuffled_inputs[train_samples + validation_samples:]
test_targets = shuffled_targets[train_samples + validation_samples:]

In [ ]:
##Aqui podemos verificar se nossas proporções estão coerentes, as porcentagens devem sempre estar entorno de 50%
##Vale lembrar que estes número mudam toda vez que se der run na cédula de shuffle do nossos dados
print(train_samples, np.sum(train_targets)/train_samples)
print(validation_samples, np.sum(validation_targets)/validation_samples)
print(test_samples, np.sum(test_targets)/test_samples)

3579 0.502374965074043
447 0.46756152125279643
448 0.5133928571428571


#Assim, terminamos todos o pré processamento dos nossos dados, e podemos salva-los em um arquivo externo e usar para treinar nosso modelo 

In [ ]:
np.savez('Audiobooks_train_data', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_validation_data', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_test_data', inputs = test_inputs, targets = test_targets)